# Test acquiring the geophone data from SAGE DMC while removing only the instrument sensitivity 

In [1]:
# Import modules

from obspy.clients.fdsn import Client
from obspy.io.sac.sactrace import SACTrace
from obspy import read_inventory
from obspy import UTCDateTime

from os import makedirs, walk, rmdir
from os.path import join, exists
from re import search
import time

from utils_basic import NETWORK

In [2]:
# Inputs 
starttime0 = UTCDateTime("2020-01-16T00:00:00")
endtime0 = UTCDateTime("2020-01-17T06:00:00")

rootdir = "/Volumes/OmanData/data/geophones_new"

# Time window length for each file in second
length_twin = 3600

In [3]:
# Compute the time period for each SAC file

numwin = round((endtime0-starttime0)/length_twin)
starttimes_twin = []
print('In total ',format(numwin, 'd'),' time periods for each station to download')
for ind in range(numwin):
	starttime = starttime0+ind*length_twin
	starttimes_twin.append(starttime)

In total  30  time periods for each station to download


In [4]:
# Get the station inventory
print('Reading the station inventory...')
inpath = join(rootdir, "station_metadata.xml")

if not exists(inpath):
	print(f"The station metadata does not exist! Quit!")
	raise

inventory = read_inventory(inpath, format="STATIONXML")
stations = inventory.get_contents()['stations']

for i, station in enumerate(stations):
    station = search(r'\((.*?)\)', station).group(1)
    stations[i] = station

Reading the station inventory...


In [5]:
# Get the data for each time window and each station
client = Client("IRIS")
network = NETWORK

# Loop over the time windows
for starttime in starttimes_twin:
    endtime = starttime + length_twin
    timestr = starttime.strftime("%Y-%m-%d-%H")
    print(f"Getting the data for {timestr}...")

    # Loop over the stations
    for station in stations:
        print(f"Getting the data for station {station}...")

        # Get the data
        try:
            stream = client.get_waveforms(network, station, "*", "GH*", starttime, endtime)
        except:
            print(f"Failed to get the data for station {station} at {timestr}!")
            continue

        # Check the data
        if len(stream) != 3:
            print(f"Incorrect number of channels! Skip the station {station} at {timestr}!")
            continue 

        print("Channels acquired:")
        for trace in stream:
            id = trace.id
            print(id)
        print("\n")

        # Remove instramental sensitivity
        print("Removing the instrumental sensitivity...")
        stream.remove_sensitivity(inventory=inventory)

        # Change the precision from float64 to float32
        for trace in stream:
            trace.data = trace.data.astype('float32')

        ## Save the data
        outdir = join(rootdir, timestr)
        if not exists(outdir):
            makedirs(outdir)
            print(f"Create the directory {outdir}...")
        
        location = stream[0].stats.location
        filename_out = f"{network}.{station}.{location}.mseed"
        outpath = join(outdir, filename_out)
        stream.write(outpath, format="MSEED")

        print(f"Data saved to {outpath}!")

        
        

Getting the data for 2020-01-16-00...
Getting the data for station A01...
Channels acquired:
7F.A01..GH1
7F.A01..GH2
7F.A01..GHZ


Removing the instrumental sensitivity...
Data saved to /Volumes/OmanData/data/geophones_new/2020-01-16-00/7F.A01..mseed!
Getting the data for station A02...


/opt/homebrew/Caskroom/miniforge/base/envs/obspy/lib/python3.11/site-packages/obspy/io/mseed/core.py:770: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)


Channels acquired:
7F.A02..GH1
7F.A02..GH2
7F.A02..GHZ


Removing the instrumental sensitivity...
Data saved to /Volumes/OmanData/data/geophones_new/2020-01-16-00/7F.A02..mseed!
Getting the data for station A03...
Channels acquired:
7F.A03..GH1
7F.A03..GH2
7F.A03..GHZ


Removing the instrumental sensitivity...
Data saved to /Volumes/OmanData/data/geophones_new/2020-01-16-00/7F.A03..mseed!
Getting the data for station A04...
Failed to get the data for station A04 at 2020-01-16-00!
Getting the data for station A05...
Channels acquired:
7F.A05..GH1
7F.A05..GH2
7F.A05..GHZ


Removing the instrumental sensitivity...
Data saved to /Volumes/OmanData/data/geophones_new/2020-01-16-00/7F.A05..mseed!
Getting the data for station A06...
